In [1]:
!pip install scikit-learn
import pandas as pd
from sklearn.ensemble import StackingClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from sklearn import model_selection, datasets, linear_model, metrics

### Ансамбль методов для классификации

### Стекинг

### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html

#### Обработка датасета

# Классификация

In [2]:
credit_df = pd.read_csv('../data/Credit_risk_preprocessed.csv')


In [3]:
credit_df["default"].unique()
x = credit_df.iloc[:,:-1].values
y = credit_df.iloc[:,4].values
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(x, y, test_size = 0.3, shuffle=False)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train_cls)

X_train_cls = scaler.transform(X_train_cls)
X_test_cls = scaler.transform(X_test_cls)

### Стекинг без подбора гиперпараметров для него и базовых алгоритмов

In [4]:
estimators = [('lr', LogisticRegression()), ('dt', DecisionTreeClassifier())]
modelClf1 = StackingClassifier(estimators=estimators)

modelClf1.fit(X_train_cls, y_train_cls)
print('Стекинг для логичтической регрессии без гиперпараметров:',modelClf1.score(X_test_cls, y_test_cls))

0.9666666666666667


#### Подбор гиперпараметров для базового алгоритма дерева решений

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn import tree
parameters = {'max_depth':[1, 2, 3, 4, 5], 'min_samples_leaf': range (1, 8) ,'min_samples_split': range (2,10,2)}
dtc = tree.DecisionTreeClassifier()
grid1_ = GridSearchCV(dtc, parameters)

In [6]:
grid1_.fit(X_train_cls, y_train_cls)
print(grid1_.best_params_)
print(grid1_.best_estimator_)

{'max_depth': 5, 'min_samples_leaf': 3, 'min_samples_split': 2}
DecisionTreeClassifier(max_depth=5, min_samples_leaf=3)


In [7]:
tr1 = tree.DecisionTreeClassifier(max_depth = 3, min_samples_leaf = 1, min_samples_split = 2)
tr1 = tr1.fit(X_train_cls, y_train_cls)
y_pred1 = tr1.predict(X_test_cls)
print('Стекинг для дерева решений с подбором гиперпараметров:',tr1.score(X_test_cls, y_test_cls))

0.9416666666666667


#### Подбор гиперпараметров для базового алгоритма логистической регрессии

In [8]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import tree
parameters = {'penalty': ('l1', 'l2', 'elasticnet', 'none'),'solver' : ['liblinear', 'sag', 'saga'], 'C': np.logspace(-4, 4, 20) }
lr = LogisticRegression()
grid2_ = GridSearchCV(lr, parameters)
grid2_.fit(X_train_cls, y_train_cls)

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                         'penalty': ('l1', 'l2', 'elasticnet', 'none'),
                         'solver': ['liblinear', 'sag', 'saga']})

In [9]:
print(grid2_.best_params_)
print(grid2_.best_estimator_)

{'C': 0.23357214690901212, 'penalty': 'l2', 'solver': 'liblinear'}
LogisticRegression(C=0.23357214690901212, solver='liblinear')


In [10]:
lr1 = LogisticRegression(C=1.623776739188721, penalty='l1', solver='liblinear')
lr1.fit(X_train_cls, y_train_cls)
y_pred = lr1.predict(X_test_cls)
print('Стекинг для логистической регресии с подбором гипер параметров:',lr1.score(X_test_cls, y_test_cls))

0.945


## Бэггинг

### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html

### Бэггинг без подбора гиперпараметров для него и базовых алгоритмов

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
modelClf3 = BaggingClassifier(base_estimator=LogisticRegression())

modelClf3.fit(X_train_cls, y_train_cls)
print('Бэггинг с базовым алгоритмом "Логистическая регрессия":',modelClf3.score(X_test_cls, y_test_cls))

Бэггинг с базовым алгоритмом "Логистическая регрессия": 0.94


In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
modelClf4 = BaggingClassifier(base_estimator=DecisionTreeClassifier())

modelClf4.fit(X_train_cls, y_train_cls)
print('Бэггинг с базовым алгоритмом "Дерево решений":',modelClf4.score(X_test_cls, y_test_cls))

Бэггинг с базовым алгоритмом "Дерево решений": 0.9933333333333333


In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
modelClf5 = BaggingClassifier(base_estimator=LogisticRegression(C=0.615848211066026, penalty='l1', solver='liblinear'))

modelClf5.fit(X_train_cls, y_train_cls)
print('Бэггинг с базовым алгоритмом "Логистическая регрессия с подбором гиперпараметров":',modelClf5.score(X_test_cls, y_test_cls))

Бэггинг с базовым алгоритмом "Логистическая регрессия с подбором гиперпараметров": 0.9416666666666667


## Адаптивный бустинг

### Документация 
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier

### Адаптивный бустинг без подбора гиперпараметров для него и базовых алгоритмов

In [19]:
from sklearn.ensemble import AdaBoostClassifier

In [20]:
modelClf7 = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
modelClf7.fit(X_train_cls, y_train_cls)
print('Адаптивный бустинг для дерева решений без гиперпараметров:',modelClf7.score(X_test_cls, y_test_cls))

0.9583333333333334


#### Подбор гиперпараметров для бустинга

In [21]:
parameters = {
              'n_estimators':[50,60,70],
              'algorithm': ['SAMME', 'SAMME.R'],
             }
abc1 = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
grid5_ = GridSearchCV(abc1, parameters)

In [22]:
grid5_.fit(X_train_cls, y_train_cls)
print(grid5_.best_params_)
print(grid5_.best_estimator_)

{'algorithm': 'SAMME', 'n_estimators': 60}
AdaBoostClassifier(algorithm='SAMME', base_estimator=DecisionTreeClassifier(),
                   n_estimators=60)


In [23]:
abc2 = AdaBoostClassifier(algorithm='SAMME', base_estimator=DecisionTreeClassifier(),
                   n_estimators=70)
abc2.fit(X_train_cls, y_train_cls)
print('Адаптивный бустинг для дерева решений с гиперпараметрами', abc2.score(X_test_cls, y_test_cls))

0.955


## Градиентный бустинг

### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

In [24]:
from sklearn.ensemble import GradientBoostingClassifier

In [25]:
modelClf8 = GradientBoostingClassifier()
modelClf8.fit(X_train_cls, y_train_cls)
print(modelClf8.score(X_test_cls, y_test_cls))

0.99


#### Подбор гиперпараметоров для градиентного бустинга

In [26]:
parameters = {
              'n_estimators':[100,150,200,300],
              'max_depth':[2,3,4,5],
              'criterion':['friedman_mse', 'squared_error'],
             }
gbc1 = GradientBoostingClassifier()
grid6_ = GridSearchCV(gbc1, parameters)

In [27]:
grid6_.fit(X_train_cls, y_train_cls)
print(grid6_.best_params_)
print(grid6_.best_estimator_)

{'criterion': 'friedman_mse', 'max_depth': 2, 'n_estimators': 150}
GradientBoostingClassifier(max_depth=2, n_estimators=150)


In [28]:
gbc2 = GradientBoostingClassifier(max_depth=2,criterion='friedman_mse', n_estimators = 100)
gbc2.fit(X_train_cls, y_train_cls)
print(gbc2.score(X_test_cls, y_test_cls))

0.9916666666666667


### Ансамбль методов для регрессии

### Стекинг без подбора гиперпараметров для него и базовых алгоритмов

#### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html#sklearn.ensemble.StackingRegressor

In [29]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [30]:
student_df = pd.read_csv('../data/student_math_preprocessed.csv')

student_df["useAlc"].unique()
x = credit_df.iloc[:,:-1].values
y = credit_df.iloc[:,4].values
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(x, y,  test_size = 0.2, random_state=0)

In [31]:
estimators = [
    ('dt', DecisionTreeRegressor(random_state=42)),
     ('lr', linear_model.LinearRegression())]
modelClf10 = StackingRegressor(estimators=estimators)

modelClf10.fit(X_train_reg, y_train_reg)
print(modelClf10.score(X_test_reg, y_test_reg))

0.8171296739083087


### Бэггинг без подбора гиперпараметров

#### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html#sklearn.ensemble.BaggingRegressor

In [32]:
from sklearn.ensemble import BaggingRegressor
modelClf12 = BaggingRegressor()
modelClf12.fit(X_train_reg, y_train_reg)
print(modelClf12.score(X_test_reg, y_test_reg))

0.8869599415697511


#### Подбор гиперпараметров для бэггинга

In [33]:
parameters = {
              'n_estimators':[2,3,4,5,6,8,9,10,20,15],
              'max_samples':[1.0,2.0,3.0],
              #'bootstrap':['True','False'],
              #'bootstrap_features':['True','False'],
              #'oob_score':['True','False']
             }
bgr2 = BaggingRegressor(base_estimator=LinearRegression())
grid8_ = GridSearchCV(bgr2, parameters)

In [34]:
grid8_.fit(X_train_reg, y_train_reg)
print(grid8_.best_params_)
print(grid8_.best_estimator_)

{'max_samples': 1.0, 'n_estimators': 8}
BaggingRegressor(base_estimator=LinearRegression(), n_estimators=8)


In [35]:
modelClf14 = BaggingRegressor(base_estimator = DecisionTreeRegressor(),max_samples=1.0, n_estimators=20)
modelClf14.fit(X_train_reg, y_train_reg)
print(modelClf14.score(X_test_reg, y_test_reg))

0.9090398336985224


### Бустинг без подбора гиперпараметров

#### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor

In [36]:
from sklearn.ensemble import AdaBoostRegressor
modelClf15 = AdaBoostRegressor()
modelClf15.fit(X_train_reg,y_train_reg)
print(modelClf15.score(X_test_reg, y_test_reg))

0.7306102765768925


#### Градиентный бустинг без подбора гиперпараметров

#### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor

In [37]:
from sklearn.ensemble import GradientBoostingRegressor
modelClf17 = GradientBoostingRegressor()
modelClf17.fit(X_train_reg, y_train_reg)
print(modelClf17.score(X_test_reg, y_test_reg))

0.811438315363526


## Catboost

#### Открытый код 
https://github.com/catboost

In [38]:
!pip install catboost

In [39]:
def print_classification_model_metrics(estimator, y_test, y_pred):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(estimator.score(X_test_cls, y_test))

from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import classification_report, confusion_matrix
cbc = CatBoostClassifier(learning_rate=0.15).fit(X_train_cls, y_train_cls)
print_classification_model_metrics(cbc, y_test_cls, cbc.predict(X_test_cls))

0:	learn: 0.4692953	total: 165ms	remaining: 2m 44s
1:	learn: 0.3302720	total: 167ms	remaining: 1m 23s
2:	learn: 0.2408309	total: 169ms	remaining: 56.2s
3:	learn: 0.1745150	total: 172ms	remaining: 42.9s
4:	learn: 0.1420410	total: 175ms	remaining: 34.8s
5:	learn: 0.1235028	total: 178ms	remaining: 29.5s
6:	learn: 0.1076271	total: 180ms	remaining: 25.5s
7:	learn: 0.0933984	total: 182ms	remaining: 22.5s
8:	learn: 0.0819392	total: 184ms	remaining: 20.3s
9:	learn: 0.0757649	total: 186ms	remaining: 18.4s
10:	learn: 0.0727758	total: 188ms	remaining: 16.9s
11:	learn: 0.0714588	total: 192ms	remaining: 15.8s
12:	learn: 0.0679745	total: 195ms	remaining: 14.8s
13:	learn: 0.0628354	total: 197ms	remaining: 13.9s
14:	learn: 0.0602450	total: 199ms	remaining: 13.1s
15:	learn: 0.0558751	total: 201ms	remaining: 12.4s
16:	learn: 0.0512265	total: 203ms	remaining: 11.7s
17:	learn: 0.0505344	total: 206ms	remaining: 11.3s
18:	learn: 0.0473527	total: 209ms	remaining: 10.8s
19:	learn: 0.0444205	total: 211ms	remai

214:	learn: 0.0023245	total: 656ms	remaining: 2.39s
215:	learn: 0.0023205	total: 659ms	remaining: 2.39s
216:	learn: 0.0023001	total: 661ms	remaining: 2.39s
217:	learn: 0.0022941	total: 664ms	remaining: 2.38s
218:	learn: 0.0022941	total: 667ms	remaining: 2.38s
219:	learn: 0.0022940	total: 669ms	remaining: 2.37s
220:	learn: 0.0022933	total: 671ms	remaining: 2.37s
221:	learn: 0.0022788	total: 676ms	remaining: 2.37s
222:	learn: 0.0022788	total: 679ms	remaining: 2.37s
223:	learn: 0.0022787	total: 684ms	remaining: 2.37s
224:	learn: 0.0022787	total: 686ms	remaining: 2.36s
225:	learn: 0.0022784	total: 688ms	remaining: 2.36s
226:	learn: 0.0022783	total: 690ms	remaining: 2.35s
227:	learn: 0.0022780	total: 693ms	remaining: 2.35s
228:	learn: 0.0022765	total: 695ms	remaining: 2.34s
229:	learn: 0.0022755	total: 697ms	remaining: 2.33s
230:	learn: 0.0022577	total: 699ms	remaining: 2.33s
231:	learn: 0.0022577	total: 701ms	remaining: 2.32s
232:	learn: 0.0022577	total: 703ms	remaining: 2.31s
233:	learn: 

457:	learn: 0.0012688	total: 1.17s	remaining: 1.39s
458:	learn: 0.0012685	total: 1.18s	remaining: 1.39s
459:	learn: 0.0012685	total: 1.18s	remaining: 1.38s
460:	learn: 0.0012682	total: 1.18s	remaining: 1.38s
461:	learn: 0.0012681	total: 1.18s	remaining: 1.38s
462:	learn: 0.0012678	total: 1.19s	remaining: 1.37s
463:	learn: 0.0012676	total: 1.19s	remaining: 1.37s
464:	learn: 0.0012673	total: 1.19s	remaining: 1.37s
465:	learn: 0.0012547	total: 1.19s	remaining: 1.37s
466:	learn: 0.0012539	total: 1.19s	remaining: 1.36s
467:	learn: 0.0012537	total: 1.2s	remaining: 1.36s
468:	learn: 0.0012534	total: 1.2s	remaining: 1.36s
469:	learn: 0.0012526	total: 1.2s	remaining: 1.35s
470:	learn: 0.0012453	total: 1.2s	remaining: 1.35s
471:	learn: 0.0012445	total: 1.21s	remaining: 1.35s
472:	learn: 0.0012445	total: 1.21s	remaining: 1.35s
473:	learn: 0.0012389	total: 1.22s	remaining: 1.35s
474:	learn: 0.0012334	total: 1.22s	remaining: 1.35s
475:	learn: 0.0012334	total: 1.22s	remaining: 1.34s
476:	learn: 0.00

623:	learn: 0.0011361	total: 1.53s	remaining: 920ms
624:	learn: 0.0011338	total: 1.53s	remaining: 918ms
625:	learn: 0.0011294	total: 1.53s	remaining: 915ms
626:	learn: 0.0011185	total: 1.53s	remaining: 913ms
627:	learn: 0.0011185	total: 1.54s	remaining: 910ms
628:	learn: 0.0011185	total: 1.54s	remaining: 908ms
629:	learn: 0.0011182	total: 1.54s	remaining: 906ms
630:	learn: 0.0011137	total: 1.54s	remaining: 904ms
631:	learn: 0.0011137	total: 1.55s	remaining: 901ms
632:	learn: 0.0011134	total: 1.55s	remaining: 898ms
633:	learn: 0.0011134	total: 1.55s	remaining: 896ms
634:	learn: 0.0011133	total: 1.55s	remaining: 893ms
635:	learn: 0.0011130	total: 1.56s	remaining: 891ms
636:	learn: 0.0011130	total: 1.56s	remaining: 888ms
637:	learn: 0.0011129	total: 1.56s	remaining: 885ms
638:	learn: 0.0011128	total: 1.56s	remaining: 882ms
639:	learn: 0.0011127	total: 1.56s	remaining: 879ms
640:	learn: 0.0011127	total: 1.56s	remaining: 877ms
641:	learn: 0.0011127	total: 1.57s	remaining: 874ms
642:	learn: 

791:	learn: 0.0009699	total: 1.86s	remaining: 487ms
792:	learn: 0.0009699	total: 1.86s	remaining: 485ms
793:	learn: 0.0009698	total: 1.86s	remaining: 483ms
794:	learn: 0.0009670	total: 1.86s	remaining: 480ms
795:	learn: 0.0009670	total: 1.87s	remaining: 478ms
796:	learn: 0.0009670	total: 1.87s	remaining: 476ms
797:	learn: 0.0009661	total: 1.87s	remaining: 474ms
798:	learn: 0.0009660	total: 1.87s	remaining: 471ms
799:	learn: 0.0009660	total: 1.88s	remaining: 469ms
800:	learn: 0.0009660	total: 1.88s	remaining: 466ms
801:	learn: 0.0009659	total: 1.88s	remaining: 464ms
802:	learn: 0.0009659	total: 1.88s	remaining: 462ms
803:	learn: 0.0009659	total: 1.88s	remaining: 459ms
804:	learn: 0.0009659	total: 1.89s	remaining: 457ms
805:	learn: 0.0009659	total: 1.89s	remaining: 454ms
806:	learn: 0.0009659	total: 1.89s	remaining: 452ms
807:	learn: 0.0009659	total: 1.89s	remaining: 449ms
808:	learn: 0.0009659	total: 1.89s	remaining: 447ms
809:	learn: 0.0009639	total: 1.89s	remaining: 444ms
810:	learn: 

963:	learn: 0.0008786	total: 2.19s	remaining: 81.8ms
964:	learn: 0.0008785	total: 2.19s	remaining: 79.6ms
965:	learn: 0.0008785	total: 2.2s	remaining: 77.3ms
966:	learn: 0.0008785	total: 2.2s	remaining: 75ms
967:	learn: 0.0008785	total: 2.2s	remaining: 72.8ms
968:	learn: 0.0008785	total: 2.2s	remaining: 70.5ms
969:	learn: 0.0008784	total: 2.21s	remaining: 68.2ms
970:	learn: 0.0008784	total: 2.21s	remaining: 66ms
971:	learn: 0.0008784	total: 2.21s	remaining: 63.7ms
972:	learn: 0.0008784	total: 2.21s	remaining: 61.4ms
973:	learn: 0.0008784	total: 2.21s	remaining: 59.1ms
974:	learn: 0.0008784	total: 2.22s	remaining: 56.8ms
975:	learn: 0.0008783	total: 2.22s	remaining: 54.5ms
976:	learn: 0.0008783	total: 2.22s	remaining: 52.3ms
977:	learn: 0.0008716	total: 2.22s	remaining: 50ms
978:	learn: 0.0008716	total: 2.22s	remaining: 47.7ms
979:	learn: 0.0008715	total: 2.23s	remaining: 45.4ms
980:	learn: 0.0008709	total: 2.23s	remaining: 43.2ms
981:	learn: 0.0008709	total: 2.23s	remaining: 40.9ms
982

In [40]:
def print_regression_model_metrics(estimator, y_test, y_pred):
    print(estimator.score(x,y))
   

In [41]:
cbr = CatBoostRegressor(learning_rate=0.15).fit(X_train_reg, y_train_reg)
print_regression_model_metrics(cbr, y_test_reg, cbr.predict(X_test_reg))

0:	learn: 0.3173620	total: 1.67ms	remaining: 1.67s
1:	learn: 0.2854855	total: 3.3ms	remaining: 1.65s
2:	learn: 0.2623477	total: 4.89ms	remaining: 1.63s
3:	learn: 0.2370408	total: 6.31ms	remaining: 1.57s
4:	learn: 0.2184003	total: 7.81ms	remaining: 1.55s
5:	learn: 0.2017159	total: 9.33ms	remaining: 1.54s
6:	learn: 0.1857589	total: 10.9ms	remaining: 1.54s
7:	learn: 0.1713354	total: 13.3ms	remaining: 1.64s
8:	learn: 0.1608444	total: 15.4ms	remaining: 1.7s
9:	learn: 0.1531084	total: 17.2ms	remaining: 1.7s
10:	learn: 0.1432548	total: 18.8ms	remaining: 1.69s
11:	learn: 0.1363384	total: 20.4ms	remaining: 1.68s
12:	learn: 0.1315130	total: 21.9ms	remaining: 1.67s
13:	learn: 0.1275344	total: 23.5ms	remaining: 1.66s
14:	learn: 0.1218690	total: 25.1ms	remaining: 1.65s
15:	learn: 0.1178017	total: 26.9ms	remaining: 1.66s
16:	learn: 0.1145156	total: 28.8ms	remaining: 1.66s
17:	learn: 0.1116220	total: 30.3ms	remaining: 1.65s
18:	learn: 0.1098943	total: 31.9ms	remaining: 1.65s
19:	learn: 0.1086256	tota

199:	learn: 0.0471609	total: 342ms	remaining: 1.37s
200:	learn: 0.0470709	total: 344ms	remaining: 1.37s
201:	learn: 0.0469331	total: 346ms	remaining: 1.36s
202:	learn: 0.0467876	total: 348ms	remaining: 1.36s
203:	learn: 0.0467481	total: 349ms	remaining: 1.36s
204:	learn: 0.0462452	total: 351ms	remaining: 1.36s
205:	learn: 0.0460648	total: 353ms	remaining: 1.36s
206:	learn: 0.0460127	total: 357ms	remaining: 1.37s
207:	learn: 0.0459433	total: 359ms	remaining: 1.37s
208:	learn: 0.0458164	total: 361ms	remaining: 1.37s
209:	learn: 0.0457573	total: 363ms	remaining: 1.36s
210:	learn: 0.0456417	total: 365ms	remaining: 1.36s
211:	learn: 0.0453971	total: 366ms	remaining: 1.36s
212:	learn: 0.0452909	total: 369ms	remaining: 1.36s
213:	learn: 0.0452346	total: 371ms	remaining: 1.36s
214:	learn: 0.0451796	total: 373ms	remaining: 1.36s
215:	learn: 0.0449412	total: 375ms	remaining: 1.36s
216:	learn: 0.0447267	total: 377ms	remaining: 1.36s
217:	learn: 0.0446767	total: 379ms	remaining: 1.36s
218:	learn: 

388:	learn: 0.0266887	total: 668ms	remaining: 1.05s
389:	learn: 0.0266158	total: 670ms	remaining: 1.05s
390:	learn: 0.0265912	total: 672ms	remaining: 1.05s
391:	learn: 0.0265381	total: 674ms	remaining: 1.04s
392:	learn: 0.0264500	total: 676ms	remaining: 1.04s
393:	learn: 0.0262809	total: 678ms	remaining: 1.04s
394:	learn: 0.0261760	total: 680ms	remaining: 1.04s
395:	learn: 0.0260776	total: 683ms	remaining: 1.04s
396:	learn: 0.0260345	total: 685ms	remaining: 1.04s
397:	learn: 0.0259157	total: 687ms	remaining: 1.04s
398:	learn: 0.0258409	total: 688ms	remaining: 1.04s
399:	learn: 0.0256739	total: 690ms	remaining: 1.03s
400:	learn: 0.0256115	total: 692ms	remaining: 1.03s
401:	learn: 0.0255902	total: 693ms	remaining: 1.03s
402:	learn: 0.0255694	total: 695ms	remaining: 1.03s
403:	learn: 0.0255073	total: 697ms	remaining: 1.03s
404:	learn: 0.0254576	total: 699ms	remaining: 1.03s
405:	learn: 0.0254321	total: 701ms	remaining: 1.02s
406:	learn: 0.0254212	total: 703ms	remaining: 1.02s
407:	learn: 

580:	learn: 0.0172163	total: 1.02s	remaining: 737ms
581:	learn: 0.0171798	total: 1.02s	remaining: 737ms
582:	learn: 0.0171386	total: 1.03s	remaining: 735ms
583:	learn: 0.0170969	total: 1.03s	remaining: 733ms
584:	learn: 0.0170589	total: 1.03s	remaining: 731ms
585:	learn: 0.0170397	total: 1.03s	remaining: 731ms
586:	learn: 0.0170125	total: 1.04s	remaining: 729ms
587:	learn: 0.0169947	total: 1.04s	remaining: 730ms
588:	learn: 0.0169275	total: 1.04s	remaining: 729ms
589:	learn: 0.0168837	total: 1.05s	remaining: 728ms
590:	learn: 0.0168349	total: 1.05s	remaining: 726ms
591:	learn: 0.0167996	total: 1.05s	remaining: 725ms
592:	learn: 0.0167832	total: 1.05s	remaining: 723ms
593:	learn: 0.0167683	total: 1.05s	remaining: 721ms
594:	learn: 0.0167567	total: 1.06s	remaining: 719ms
595:	learn: 0.0167330	total: 1.06s	remaining: 717ms
596:	learn: 0.0166890	total: 1.06s	remaining: 715ms
597:	learn: 0.0166529	total: 1.06s	remaining: 713ms
598:	learn: 0.0166441	total: 1.06s	remaining: 712ms
599:	learn: 

758:	learn: 0.0122935	total: 1.34s	remaining: 427ms
759:	learn: 0.0122805	total: 1.35s	remaining: 425ms
760:	learn: 0.0122517	total: 1.35s	remaining: 424ms
761:	learn: 0.0122432	total: 1.35s	remaining: 422ms
762:	learn: 0.0122268	total: 1.35s	remaining: 420ms
763:	learn: 0.0121952	total: 1.35s	remaining: 418ms
764:	learn: 0.0121852	total: 1.36s	remaining: 417ms
765:	learn: 0.0121643	total: 1.36s	remaining: 415ms
766:	learn: 0.0121558	total: 1.36s	remaining: 413ms
767:	learn: 0.0121373	total: 1.36s	remaining: 412ms
768:	learn: 0.0121249	total: 1.36s	remaining: 410ms
769:	learn: 0.0120988	total: 1.37s	remaining: 408ms
770:	learn: 0.0120822	total: 1.37s	remaining: 406ms
771:	learn: 0.0120701	total: 1.37s	remaining: 404ms
772:	learn: 0.0120506	total: 1.37s	remaining: 403ms
773:	learn: 0.0120355	total: 1.37s	remaining: 401ms
774:	learn: 0.0120111	total: 1.38s	remaining: 399ms
775:	learn: 0.0119979	total: 1.38s	remaining: 398ms
776:	learn: 0.0119894	total: 1.38s	remaining: 396ms
777:	learn: 

939:	learn: 0.0093599	total: 1.68s	remaining: 107ms
940:	learn: 0.0093468	total: 1.68s	remaining: 105ms
941:	learn: 0.0093283	total: 1.68s	remaining: 104ms
942:	learn: 0.0093168	total: 1.69s	remaining: 102ms
943:	learn: 0.0093112	total: 1.69s	remaining: 100ms
944:	learn: 0.0092825	total: 1.69s	remaining: 98.3ms
945:	learn: 0.0092795	total: 1.69s	remaining: 96.6ms
946:	learn: 0.0092464	total: 1.69s	remaining: 94.8ms
947:	learn: 0.0092424	total: 1.7s	remaining: 93ms
948:	learn: 0.0092376	total: 1.7s	remaining: 91.3ms
949:	learn: 0.0092335	total: 1.7s	remaining: 89.5ms
950:	learn: 0.0092092	total: 1.7s	remaining: 87.7ms
951:	learn: 0.0092047	total: 1.7s	remaining: 85.9ms
952:	learn: 0.0091929	total: 1.7s	remaining: 84.1ms
953:	learn: 0.0091595	total: 1.71s	remaining: 82.3ms
954:	learn: 0.0091461	total: 1.71s	remaining: 80.5ms
955:	learn: 0.0091417	total: 1.71s	remaining: 78.7ms
956:	learn: 0.0091217	total: 1.71s	remaining: 76.9ms
957:	learn: 0.0091163	total: 1.71s	remaining: 75.2ms
958:	l